<a href="https://colab.research.google.com/github/Manju1803-a11y/Project_SIC/blob/main/literacy_rate_predction_webpage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Frontend

In [3]:
!pip install pandas gradio matplotlib

In [4]:
import pandas as pd

df = pd.read_csv("/content/literacy_rate.csv")
df.head()

,education_spending_percent_gdp,urbanization_rate,avg_income_usd,population_density,school_availability_index,teacher_student_ratio,internet_penetration,gender_equality_index,adult_education_programs,language_diversity,region_type,government_stability,literacy_rate_percent
0,4.847706,70.937794,20098.745559,503.001686,0.526940,35.269097,90.755242,NaN,NaN,Bilingual,Least Developed,High,66.516441
1,7.281657,64.538697,50606.686295,737.794798,0.524418,NaN,90.197009,0.500205,Extensive,Multilingual,Least Developed,NaN,85.431091
2,3.833110,63.533592,53415.197757,NaN,0.855385,16.078349,60.915648,0.921771,NaN,Monolingual,Developing,Low,67.063415
3,NaN,37.964468,10943.100757,343.572155,0.864194,40.721066,28.890170,0.862589,Limited,NaN,Developing,High,58.288996
4,2.697866,74.761894,26233.323359,753.260001,0.932270,38.795291,72.056473,0.682091,Extensive,NaN,Least Developed,Low,87.016369


In [5]:
%%writefile app.py
import streamlit as st
import sqlite3, hashlib, random
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# =====================
# DATABASE
# =====================

conn = sqlite3.connect("users.db", check_same_thread=False)
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS users(
    username TEXT PRIMARY KEY,
    mobile TEXT,
    password TEXT
)
""")
conn.commit()

def hash_pass(p):
    return hashlib.sha256(p.encode()).hexdigest()

otp_store = {}

def register(u,m,p):
    try:
        cur.execute("INSERT INTO users VALUES (?,?,?)",(u,m,hash_pass(p)))
        conn.commit()
        return "Registered successfully"
    except:
        return "Username already exists"

def login(u,p):
    cur.execute("SELECT password FROM users WHERE username=?",(u,))
    r = cur.fetchone()
    return r and r[0]==hash_pass(p)

def send_otp(m):
    cur.execute("SELECT username FROM users WHERE mobile=?",(m,))
    if not cur.fetchone():
        return "Mobile not found"
    otp = str(random.randint(100000,999999))
    otp_store[m]=otp
    print("OTP:",otp)
    return "OTP sent (see Colab output)"

def reset_pass(m,o,np):
    if m not in otp_store or otp_store[m]!=o:
        return "Wrong OTP"
    cur.execute("UPDATE users SET password=? WHERE mobile=?",(hash_pass(np),m))
    conn.commit()
    del otp_store[m]
    return "Password updated"

# =====================
# ML MODEL
# =====================

df = pd.DataFrame({
 "School_Enrollment":[60,70,80,90,50,65,75,85,95,55],
 "Gov_Spending":[3,4,5,6,2,3.5,4.5,5.5,6.5,2.5],
 "Urban_Population":[40,50,60,70,35,45,55,65,75,38],
 "Literacy_Rate":[62,70,78,85,55,68,75,82,88,58]
})

X=df.drop("Literacy_Rate",axis=1)
y=df["Literacy_Rate"]

model=RandomForestRegressor(n_estimators=200)
model.fit(X,y)

# =====================
# SESSION
# =====================

if "logged" not in st.session_state:
    st.session_state.logged=False

# =====================
# UI
# =====================

if not st.session_state.logged:

    st.title("🔐 Manjunath Literacy Prediction Login")

    u=st.text_input("Username")
    p=st.text_input("Password",type="password")

    if st.button("Login"):
        if login(u,p):
            st.session_state.logged=True
            st.rerun()
        else:
            st.error("Invalid credentials")

    st.subheader("Register")

    ru=st.text_input("New Username")
    rm=st.text_input("Mobile")
    rp=st.text_input("New Password",type="password")

    if st.button("Register"):
        st.success(register(ru,rm,rp))

    st.subheader("Forgot Password")

    fm=st.text_input("Registered Mobile")

    if st.button("Send OTP"):
        st.info(send_otp(fm))

    otp=st.text_input("OTP")
    newp=st.text_input("New Password",type="password")

    if st.button("Reset Password"):
        st.success(reset_pass(fm,otp,newp))

else:
    st.title("📊 Literacy Rate Prediction")

    e=st.slider("School Enrollment (%)",30,100,70)
    s=st.slider("Gov Spending (%)",1.0,8.0,4.0)
    u=st.slider("Urban Population (%)",20,90,50)

    if st.button("Predict"):
        pred=model.predict([[e,s,u]])[0]
        st.success(f"Literacy Rate: {pred:.2f}%")

    if st.button("Logout"):
        st.session_state.logged=False
        st.rerun()

Writing app.py


In [6]:
!pip install streamlit pyngrok
!streamlit run app.py &>/content/logs.txt &

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 112.1 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 7.0.0
    Uninstalling cachetools-7.0.0:
      Successfully uninstalled cachetools-7.0.0


In [7]:
from pyngrok import ngrok

# Replace 'YOUR_AUTHTOKEN' with your actual ngrok authtoken
# You can find your authtoken on the ngrok dashboard after signing up.
ngrok.set_auth_token('39YOqvQCJpTtbIhToDwQEagWsxR_6aXQ5vfcMbcKSozjaMxHD')

ngrok.kill()
public_url = ngrok.connect(8501)
print("LIVE APP:", public_url)

LIVE APP: NgrokTunnel: "https://gigi-unencouraging-malinda.ngrok-free.dev" -> "http://localhost:8501"
